In [1]:
import pandas as pd
import os
import sys
import numpy as np
from pathlib import Path
from tqdm import trange
from transformers import AutoModel, AutoTokenizer
from konlpy.tag import Mecab, Okt

In [2]:
sys.path.append(str(Path.home().joinpath("level3_nlp_finalproject-nlp-04")))
from utils.preprocessing import *

In [3]:
news = pd.read_csv(Path.cwd().parent.joinpath("Data", "News", "005930.csv"))

In [4]:
# news = preprocess_dataframe(news)
news['content'].replace('', np.nan, inplace=True)
news.dropna(subset=['content'], inplace=True)
news.reset_index(drop=True, inplace=True)

In [5]:
news

,company,title,link,writer,date,content
0,삼성전자,“지독한 중국 떼로 덤비더니” 삼성 점유율 절반 뺏아갔다,https://finance.naver.com/item/news_read.naver...,헤럴드경제,2023.05.18 22:14,중국 화웨이 폴더블폰 S포켓[헤럴드경제= 박영훈 기자] “삼성 타도 외치더니” 중...
1,삼성전자,"“반도체 시간이 온다”…야심작 출격한 삼성전자, 비밀병기는",https://finance.naver.com/item/news_read.naver...,매일경제,2023.05.18 19:02,12나노급 DDR5 D램 양산 EUV 공정 등 기술력 총동원 DDR5로의 시장 전환...
2,삼성전자,D램 격차 더 벌린 삼성전자…업계 첫 12나노급 양산,https://finance.naver.com/item/news_read.naver...,파이낸셜뉴스,2023.05.18 18:41,1초에 UHD 영화 2편 처리 가능 생산성 20% 소비전력 23% 개선 ‘챗GPT’...
3,삼성전자,"현대차에 내준 '영업이익 1위' 삼성전자, 하반기엔 되찾는다",https://finance.naver.com/item/news_read.naver...,매일경제,2023.05.18 17:46,현대차 실적 상고하저 예상 3분기 2조8200억원 그칠듯 삼성전자 하반기 본격 반등...
4,삼성전자,삼성전자 '12나노급 D램' 세계 첫 양산,https://finance.naver.com/item/news_read.naver...,매일경제,2023.05.18 17:43,반도체 업황부진 뚫을 야심작 생산성 20%·소비전력 23% ↑삼성전자가 메모리반도체...
...,...,...,...,...,...,...
2874,삼성전자,"[특징주] 삼성전자, 장중 7만원선 밑돌아…외인 매도에 1% 하락(종합...",https://finance.naver.com/item/news_read.naver...,연합뉴스,2023.07.21 15:44,(CG) [연합뉴스TV 제공](서울=연합뉴스) 송은경 기자 = 글로벌 반도체 업종 ...
2875,삼성전자,"AMD CEO ""韓 언론을 믿냐""…삼성, AMD 3나노 수주보도에 'T...",https://finance.naver.com/item/news_read.naver...,아시아경제,2023.07.21 15:32,4년 만에 대만 방문한 리사 수 AMD CEO미국 반도체 업계를 대표하는 대만계 경...
2876,삼성전자,"바닥 찍은 반도체·디스플레이… 삼성전자·LGD, 4분기 흑자전환 기대감...",https://finance.naver.com/item/news_read.naver...,조선비즈,2023.07.21 15:26,"삼성전자, 3분기부터 메모리 감산 효과 본격화 3분기 들어 D램 가격 하락폭도 안정..."
2877,삼성전자,'황금명함'만 있는 게 아니다...삼성전자 새내기 부모님 '이것' 받고...,https://finance.naver.com/item/news_read.naver...,파이낸셜뉴스,2023.07.21 14:53,"삼성전자 반도체(DS)부문 입사 예정자들에게 전달된 황금명함과 꽃다발, 경계현 사장..."


In [ ]:
# Train New Tokenizer
# def get_news(news):
# 	content = news['content']
# 	for start_idx in range(0, len(content), 1000):
# 		samples = content[start_idx:start_idx+1000]
# 		yield samples

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
# trained_tokenizer = tokenizer.train_new_from_iterator(get_news(news), vocab_size=32000)

In [ ]:
# trained_tokenizer.tokenize(news.sample(1)['content'].item())

In [ ]:
okt = Okt()

In [ ]:
company = news['company'].unique()

In [ ]:
ecopro = news[news['company'] == '에코프로']['content']

In [ ]:
samsung = news[news['company'] == '삼성전자']['content']

In [ ]:
for i in trange(len(samsung)):
	samsung[i] = " ".join(okt.nouns(samsung[i]))

In [ ]:
from krwordrank.word import summarize_with_keywords, KRWordRank

In [ ]:
wordrank_extractor = KRWordRank(min_count=10, max_length=100)

beta = 0.85
max_iter = 10
keywords, rank, graph = wordrank_extractor.extract(samsung.tolist(), beta, max_iter)

In [ ]:
for key, score in sorted(graph[5].items(), reverse=True, key=lambda x: x[1])[:100]:
	print(wordrank_extractor.int2token(key))
	print(score)

In [ ]:
samsung_keywords = summarize_with_keywords(samsung.tolist(),
								   min_count=10,
								   max_length=100,
								   beta=0.85,
								   max_iter=10,
								   verbose=False)

In [ ]:
samsung_keywords

In [ ]:
ecopro_keywords = summarize_with_keywords(ecopro.tolist(),
								   min_count=10,
								   max_length=100,
								   beta=0.85,
								   max_iter=10,
								   verbose=True)
ecopro_keywords

In [ ]:
top_k = set(samsung_keywords) - set(ecopro_keywords)

In [ ]:
t = {key : samsung_keywords[key] for key in top_k}

In [ ]:
sorted(t.items() , reverse=True, key=lambda x: x[1])